In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [22]:
def generate_board():
    board = np.random.choice([1, -1, 0], size=9)  # 1 for X, -1 for O, 0 for empty
    return board

In [23]:
def generate_dataset(num_samples=10000):
    data = []
    labels = []
    for _ in range(num_samples):
        board = generate_board()
        winner = check_winner_np(board)
        while winner is None:
            board = generate_board()
            winner = check_winner_np(board)
        data.append(board)
        labels.append(winner)
    df = pd.DataFrame(data, columns=[f'cell_{i}' for i in range(9)])
    df['result'] = labels
    return df


In [24]:
def check_winner_np(board):
    wins = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Horizontal
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Vertical
        [0, 4, 8], [2, 4, 6]              # Diagonal
    ]
    for win in wins:
        if np.sum(board[win]) == 3:
            return 1  # X wins
        elif np.sum(board[win]) == -3:
            return -1  # O wins
    if 0 not in board:
        return 0  # Draw
    return None  # Game is not over

In [25]:
df = generate_dataset(10000)
X = df.drop('result', axis=1)
y = df['result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 93.15%


In [26]:
def human_vs_model():
    board = np.zeros(9, dtype=int)  # Empty board
    turn = 1  # X starts (human)
    while True:
        print_board(board)
        if turn == 1:
            move = int(input("Your turn! Choose a position (0-8): "))
            if move < 0 or move > 8 or board[move] != 0:
                print("Invalid move! Try again.")
                continue
            board[move] = turn
        else:
            available_moves = [i for i in range(9) if board[i] == 0]
            move = model_choose_move(board, available_moves)
            board[move] = turn
            print(f"Model plays at position: {move}")
        winner = check_winner(board)
        if winner is not None:
            print_board(board)
            if winner == 1:
                print("X wins!")
            elif winner == -1:
                print("O wins!")
            else:
                print("It's a draw!")
            break
        turn = 1 - turn  # Switch turns

       

In [27]:
def print_board(board):
    symbols = {0: '.', 1: 'X', -1: 'O'}
    for i in range(0, 9, 3):
        print(' '.join(symbols[board[i]] for i in range(i, i+3)))
    print()

    

In [28]:
def model_choose_move(board, available_moves):
    return np.random.choice(available_moves)
